In [20]:
import pandas as pd
import numpy as np
import re

In [21]:
data=pd.read_excel('赛题1数据材料\赛题1数据集\上市信息财务信息-利润表.xlsx')

In [22]:
data

,企业编号,标题,日期,营业收入(元),营业成本(元),销售费用(元),财务费用(元),管理费用(元),资产减值损失(元),投资收益(元),营业利润(元),利润总额(元),所得税(元),归属母公司所有者净利润(元)
0,1001,利润表,2018-09-30,127109000000,NaN,NaN,NaN,30421000000,44072000000,11116000000,50918000000,50893000000,7229000000,43034000000
1,1001,利润表,2018-06-30,82256000000,NaN,NaN,NaN,19971000000,26929000000,9382000000,34164000000,34161000000,5261000000,28432000000
2,1001,利润表,2018-03-31,39629000000,NaN,NaN,NaN,9680000000,12173000000,3696000000,17105000000,17106000000,2647000000,14266000000
3,1001,利润表,2017-12-31,168619000000,NaN,NaN,NaN,41047000000,55285000000,13585000000,70275000000,69828000000,14826000000,54046000000
4,1001,利润表,2017-09-30,125471000000,NaN,NaN,NaN,29275000000,40145000000,8391000000,54621000000,54743000000,12347000000,41305000000
5,1001,利润表,2017-06-30,84045000000,NaN,NaN,NaN,19236000000,27170000000,6138000000,36671000000,36751000000,8229000000,27618000000
6,1002,利润表,2018-09-30,5803394924.71,4145927467.38,21521618.9,72485135.28,337278554.46,11890361.38,35859554.47,1247170227.63,1252524462.81,312893396.74,914633402.37
7,1002,利润表,2018-06-30,3720615775.06,2524410387.14,16058592.17,32046137.95,188584639.56,31814002.69,17785221.48,951928844.35,954963763.65,239172814.25,700857214.35
8,1002,利润表,2018-03-31,1723798620.17,1039236420.46,13675219.71,-5158955.5,67359452.91,11965766.7,8784291.9,594946239.92,596388971.82,159952657.62,431843540.39
9,1002,利润表,2017-12-31,6761550874.37,4018885671.11,88346033.54,38668629.07,423106162.89,18778746.5,45618283.41,2164936622.04,2165782070.2,564537893.88,1564238738.86


In [23]:
#将单位全部统一为以元为单位
def convert(cell):
    pattern1=(r'(-?\d+\.?\d+)亿')
    pattern2=(r'(-?\d+\.?\d+)万亿')
    pattern3=(r'(-?\d+\.?\d+)万')
    pattern4=(r'(-?\d+\.?\d+)')
    if re.findall(pattern1,cell)!=[]:
        cell=float(re.findall(pattern1, cell)[0])/10000
    elif re.findall(pattern2,cell)!=[]:
        cell=float(re.findall(pattern2, cell)[0])
    elif re.findall(pattern3,cell)!=[]:
        cell=float(re.findall(pattern3, cell)[0])/10000000
    elif re.findall(pattern4,cell)!=[]:
        cell=float(re.findall(pattern4, cell)[0])/100000000000
    elif cell=='nan'or cell=='--':
        cell=np.nan
    else: cell=float(cell)
    return cell

In [24]:
data.iloc[:,3:]=data.iloc[:,3:].applymap(str).applymap(convert)

In [25]:
data

,企业编号,标题,日期,营业收入(元),营业成本(元),销售费用(元),财务费用(元),管理费用(元),资产减值损失(元),投资收益(元),营业利润(元),利润总额(元),所得税(元),归属母公司所有者净利润(元)
0,1001,利润表,2018-09-30,1.271090,NaN,NaN,NaN,0.304210,0.440720,1.111600e-01,0.509180,0.508930,0.072290,0.430340
1,1001,利润表,2018-06-30,0.822560,NaN,NaN,NaN,0.199710,0.269290,9.382000e-02,0.341640,0.341610,0.052610,0.284320
2,1001,利润表,2018-03-31,0.396290,NaN,NaN,NaN,0.096800,0.121730,3.696000e-02,0.171050,0.171060,0.026470,0.142660
3,1001,利润表,2017-12-31,1.686190,NaN,NaN,NaN,0.410470,0.552850,1.358500e-01,0.702750,0.698280,0.148260,0.540460
4,1001,利润表,2017-09-30,1.254710,NaN,NaN,NaN,0.292750,0.401450,8.391000e-02,0.546210,0.547430,0.123470,0.413050
5,1001,利润表,2017-06-30,0.840450,NaN,NaN,NaN,0.192360,0.271700,6.138000e-02,0.366710,0.367510,0.082290,0.276180
6,1002,利润表,2018-09-30,0.058034,0.041459,0.000215,0.000725,0.003373,0.000119,3.585955e-04,0.012472,0.012525,0.003129,0.009146
7,1002,利润表,2018-06-30,0.037206,0.025244,0.000161,0.000320,0.001886,0.000318,1.778522e-04,0.009519,0.009550,0.002392,0.007009
8,1002,利润表,2018-03-31,0.017238,0.010392,0.000137,-0.000052,0.000674,0.000120,8.784292e-05,0.005949,0.005964,0.001600,0.004318
9,1002,利润表,2017-12-31,0.067616,0.040189,0.000883,0.000387,0.004231,0.000188,4.561828e-04,0.021649,0.021658,0.005645,0.015642


In [26]:
#查看出现较多的时间
data['日期'].value_counts()

2017-09-30    3201
2017-06-30    3146
2017-12-31    3133
2018-03-31    3104
2018-06-30    3065
2018-09-30    3031
2016-12-31     202
2017-03-31     176
2016-09-30     169
2016-06-30     138
2015-12-31      89
2016-03-31      77
2018-12-31      46
2014-12-31       5
15-12-31         4
16-03-31         4
16-06-30         4
15-06-30         4
15-09-30         4
2013-12-31       3
15-03-31         3
16-09-30         2
14-12-31         2
16-12-31         2
14-09-30         2
2012-12-31       1
2011-12-31       1
17-03-31         1
2015-06-30       1
2015-09-30       1
Name: 日期, dtype: int64

In [27]:
#补全时间，规整时间的格式
pattern='\d\d\d\d-\d\d-\d\d'
for i in range(len(data)):
    if re.findall(pattern,data.iloc[i,2])==[]:
        data.iloc[i,2]='20'+data.iloc[i,2]

In [28]:
#去除企业编号与日期值同时相同的行，保留第一次出现的行
data.drop_duplicates(subset=['企业编号','日期'],keep='first',inplace=True)

In [29]:
#再次查看出现较多的时间
data['日期'].value_counts()

2017-09-30    2982
2017-12-31    2980
2018-06-30    2974
2018-09-30    2973
2018-03-31    2957
2017-06-30    2928
2016-12-31     163
2017-03-31     136
2016-09-30     131
2016-06-30     109
2015-12-31      78
2016-03-31      66
2018-12-31      45
2014-12-31       7
2015-09-30       5
2015-06-30       5
2013-12-31       3
2015-03-31       3
2014-09-30       2
2012-12-31       1
2011-12-31       1
Name: 日期, dtype: int64

In [30]:
#将数量较少的日期全部丢弃
timelist=list(['2017-09-30','2017-12-31','2018-06-30','2018-09-30','2017-06-30','2018-03-31'])
data=data[data['日期'].isin(timelist)]
data

,企业编号,标题,日期,营业收入(元),营业成本(元),销售费用(元),财务费用(元),管理费用(元),资产减值损失(元),投资收益(元),营业利润(元),利润总额(元),所得税(元),归属母公司所有者净利润(元)
0,1001,利润表,2018-09-30,1.271090,NaN,NaN,NaN,0.304210,0.440720,1.111600e-01,0.509180,0.508930,0.072290,0.430340
1,1001,利润表,2018-06-30,0.822560,NaN,NaN,NaN,0.199710,0.269290,9.382000e-02,0.341640,0.341610,0.052610,0.284320
2,1001,利润表,2018-03-31,0.396290,NaN,NaN,NaN,0.096800,0.121730,3.696000e-02,0.171050,0.171060,0.026470,0.142660
3,1001,利润表,2017-12-31,1.686190,NaN,NaN,NaN,0.410470,0.552850,1.358500e-01,0.702750,0.698280,0.148260,0.540460
4,1001,利润表,2017-09-30,1.254710,NaN,NaN,NaN,0.292750,0.401450,8.391000e-02,0.546210,0.547430,0.123470,0.413050
5,1001,利润表,2017-06-30,0.840450,NaN,NaN,NaN,0.192360,0.271700,6.138000e-02,0.366710,0.367510,0.082290,0.276180
6,1002,利润表,2018-09-30,0.058034,0.041459,0.000215,0.000725,0.003373,0.000119,3.585955e-04,0.012472,0.012525,0.003129,0.009146
7,1002,利润表,2018-06-30,0.037206,0.025244,0.000161,0.000320,0.001886,0.000318,1.778522e-04,0.009519,0.009550,0.002392,0.007009
8,1002,利润表,2018-03-31,0.017238,0.010392,0.000137,-0.000052,0.000674,0.000120,8.784292e-05,0.005949,0.005964,0.001600,0.004318
9,1002,利润表,2017-12-31,0.067616,0.040189,0.000883,0.000387,0.004231,0.000188,4.561828e-04,0.021649,0.021658,0.005645,0.015642


In [31]:
data.sort_values(by=['企业编号','日期'],inplace=True)

D:\ANACONDA2\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [32]:
data

,企业编号,标题,日期,营业收入(元),营业成本(元),销售费用(元),财务费用(元),管理费用(元),资产减值损失(元),投资收益(元),营业利润(元),利润总额(元),所得税(元),归属母公司所有者净利润(元)
5,1001,利润表,2017-06-30,0.840450,NaN,NaN,NaN,0.192360,0.271700,6.138000e-02,0.366710,0.367510,0.082290,0.276180
4,1001,利润表,2017-09-30,1.254710,NaN,NaN,NaN,0.292750,0.401450,8.391000e-02,0.546210,0.547430,0.123470,0.413050
3,1001,利润表,2017-12-31,1.686190,NaN,NaN,NaN,0.410470,0.552850,1.358500e-01,0.702750,0.698280,0.148260,0.540460
2,1001,利润表,2018-03-31,0.396290,NaN,NaN,NaN,0.096800,0.121730,3.696000e-02,0.171050,0.171060,0.026470,0.142660
1,1001,利润表,2018-06-30,0.822560,NaN,NaN,NaN,0.199710,0.269290,9.382000e-02,0.341640,0.341610,0.052610,0.284320
0,1001,利润表,2018-09-30,1.271090,NaN,NaN,NaN,0.304210,0.440720,1.111600e-01,0.509180,0.508930,0.072290,0.430340
11,1002,利润表,2017-06-30,0.032137,0.019455,0.000356,-0.000031,0.001887,0.000023,1.882209e-04,0.010504,0.010512,0.002648,0.007803
10,1002,利润表,2017-09-30,0.049185,0.030027,0.000519,-0.000023,0.002846,0.000083,3.722918e-04,0.016070,0.015773,0.003972,0.011987
9,1002,利润表,2017-12-31,0.067616,0.040189,0.000883,0.000387,0.004231,0.000188,4.561828e-04,0.021649,0.021658,0.005645,0.015642
8,1002,利润表,2018-03-31,0.017238,0.010392,0.000137,-0.000052,0.000674,0.000120,8.784292e-05,0.005949,0.005964,0.001600,0.004318


In [33]:
#查看排序结果是否达到目标效果
data.columns=pd.Index(['企业编号', '标题', '日期', '$营业收入(元)', '$营业成本(元)', '$销售费用(元)', '$财务费用(元)',
       '$管理费用(元)', '$资产减值损失(元)', '$投资收益(元)', '$营业利润(元)', '$利润总额(元)', '$所得税(元)',
       '$归属母公司所有者净利润(元)'])

In [34]:
data

,企业编号,标题,日期,$营业收入(元),$营业成本(元),$销售费用(元),$财务费用(元),$管理费用(元),$资产减值损失(元),$投资收益(元),$营业利润(元),$利润总额(元),$所得税(元),$归属母公司所有者净利润(元)
5,1001,利润表,2017-06-30,0.840450,NaN,NaN,NaN,0.192360,0.271700,6.138000e-02,0.366710,0.367510,0.082290,0.276180
4,1001,利润表,2017-09-30,1.254710,NaN,NaN,NaN,0.292750,0.401450,8.391000e-02,0.546210,0.547430,0.123470,0.413050
3,1001,利润表,2017-12-31,1.686190,NaN,NaN,NaN,0.410470,0.552850,1.358500e-01,0.702750,0.698280,0.148260,0.540460
2,1001,利润表,2018-03-31,0.396290,NaN,NaN,NaN,0.096800,0.121730,3.696000e-02,0.171050,0.171060,0.026470,0.142660
1,1001,利润表,2018-06-30,0.822560,NaN,NaN,NaN,0.199710,0.269290,9.382000e-02,0.341640,0.341610,0.052610,0.284320
0,1001,利润表,2018-09-30,1.271090,NaN,NaN,NaN,0.304210,0.440720,1.111600e-01,0.509180,0.508930,0.072290,0.430340
11,1002,利润表,2017-06-30,0.032137,0.019455,0.000356,-0.000031,0.001887,0.000023,1.882209e-04,0.010504,0.010512,0.002648,0.007803
10,1002,利润表,2017-09-30,0.049185,0.030027,0.000519,-0.000023,0.002846,0.000083,3.722918e-04,0.016070,0.015773,0.003972,0.011987
9,1002,利润表,2017-12-31,0.067616,0.040189,0.000883,0.000387,0.004231,0.000188,4.561828e-04,0.021649,0.021658,0.005645,0.015642
8,1002,利润表,2018-03-31,0.017238,0.010392,0.000137,-0.000052,0.000674,0.000120,8.784292e-05,0.005949,0.005964,0.001600,0.004318


In [35]:
#'营业收入(元)',营业成本(元),
# data['营业成本(元)'].describe()
# data['营业成本(元)'].sort_values(ascending=False)

In [36]:
data=data.set_index(['企业编号','日期'])
data=data.unstack().reset_index().set_index('企业编号').T.drop('标题')
data

企业编号                           1001         1002         1003         1004  \
                日期                                                           
$营业收入(元)        2017-06-30  0.84045    0.0321366    0.0814589    0.0125497   
                2017-09-30  1.25471    0.0491849     0.129964    0.0199218   
                2017-12-31  1.68619    0.0676155     0.183009    0.0276646   
                2018-03-31  0.39629     0.017238    0.0245778    0.0077753   
                2018-06-30  0.82256    0.0372062    0.0578917    0.0155212   
                2018-09-30  1.27109    0.0580339    0.0924283    0.0233654   
$营业成本(元)        2017-06-30      NaN    0.0194546    0.0724443   0.00578449   
                2017-09-30      NaN    0.0300267     0.115687   0.00942475   
                2017-12-31      NaN    0.0401889     0.163431     0.013396   
                2018-03-31      NaN    0.0103924     0.020959   0.00372713   
                2018-06-30      NaN    0.0252441     0.050713   0.00728342   
                2018-09-30      NaN    0.0414593    0.0819203    0.0109525   
$销售费用(元)        2017-06-30      NaN  0.000355517   0.00424572  0.000419261   
                2017-09-30      NaN  0.000518927   0.00765363  0.000602727   
                2017-12-31      NaN   0.00088346     0.010186  0.000947737   
                2018-03-31      NaN  0.000136752   0.00122953  0.000104714   
                2018-06-30      NaN  0.000160586   0.00272415  0.000276326   
                2018-09-30      NaN  0.000215216   0.00417515  0.000460654   
$财务费用(元)        2017-06-30      NaN -3.11722e-05 -0.000201935  0.000586675   
                2017-09-30      NaN -2.30314e-05 -7.60198e-05   0.00101251   
                2017-12-31      NaN  0.000386686 -0.000114174   0.00141558   
                2018-03-31      NaN -5.15896e-05 -3.21156e-05  0.000373582   
                2018-06-30      NaN  0.000320461 -0.000160898  0.000710632   
                2018-09-30      NaN  0.000724851 -0.000194951   0.00108634   
$管理费用(元)        2017-06-30  0.19236   0.00188651   0.00577859  0.000632755   
                2017-09-30  0.29275   0.00284589   0.00881906  0.000837301   
                2017-12-31  0.41047   0.00423106    0.0110121   0.00110857   
                2018-03-31   0.0968  0.000673595   0.00199652  0.000204399   
                2018-06-30  0.19971   0.00188585    0.0041144  0.000410844   
                2018-09-30  0.30421   0.00337279   0.00646652  0.000616354   
...                             ...          ...          ...          ...   
$投资收益(元)        2017-06-30  0.06138  0.000188221   0.00265996  2.42543e-05   
                2017-09-30  0.08391  0.000372292   0.00380402  5.51164e-05   
                2017-12-31  0.13585  0.000456183   0.00444671  8.66791e-05   
                2018-03-31  0.03696  8.78429e-05   0.00107747  2.81326e-05   
                2018-06-30  0.09382  0.000177852   0.00293019  3.96617e-05   
                2018-09-30  0.11116  0.000358596   0.00456705  6.66262e-05   
$营业利润(元)        2017-06-30  0.36671     0.010504  5.77859e-05   0.00393754   
                2017-09-30  0.54621    0.0160703  -0.00167333   0.00623709   
                2017-12-31  0.70275    0.0216494  -0.00089908   0.00827209   
                2018-03-31  0.17105   0.00594946    0.0013979   0.00268932   
                2018-06-30  0.34164   0.00951929   0.00290976   0.00550701   
                2018-09-30  0.50918    0.0124717   0.00384793   0.00821564   
$利润总额(元)        2017-06-30  0.36751    0.0105121 -1.33306e-06    0.0039342   
                2017-09-30  0.54743    0.0157729  -0.00196006    0.0063644   
                2017-12-31  0.69828    0.0216578  -0.00118883   0.00849004   
                2018-03-31  0.17106   0.00596389   0.00139051   0.00268868   
                2018-06-30  0.34161   0.00954964   0.00290272   0.00538599   
                2018-09-30  0.50893    0.0125252    0.0037417   0.00786801   
$所得税(元)         2017-0

In [37]:
data=data.reindex(columns=list(range(1001,4001)))
data.to_pickle('上市信息财务信息-利润表.pickle')
data.index

MultiIndex(levels=[['标题', '$营业收入(元)', '$营业成本(元)', '$销售费用(元)', '$财务费用(元)', '$管理费用(元)', '$资产减值损失(元)', '$投资收益(元)', '$营业利润(元)', '$利润总额(元)', '$所得税(元)', '$归属母公司所有者净利润(元)', '企业编号'], ['2017-06-30', '2017-09-30', '2017-12-31', '2018-03-31', '2018-06-30', '2018-09-30', '']],
           labels=[[1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 3, 3, 3, 3, 3, 3, 4, 4, 4, 4, 4, 4, 5, 5, 5, 5, 5, 5, 6, 6, 6, 6, 6, 6, 7, 7, 7, 7, 7, 7, 8, 8, 8, 8, 8, 8, 9, 9, 9, 9, 9, 9, 10, 10, 10, 10, 10, 10, 11, 11, 11, 11, 11, 11], [0, 1, 2, 3, 4, 5, 0, 1, 2, 3, 4, 5, 0, 1, 2, 3, 4, 5, 0, 1, 2, 3, 4, 5, 0, 1, 2, 3, 4, 5, 0, 1, 2, 3, 4, 5, 0, 1, 2, 3, 4, 5, 0, 1, 2, 3, 4, 5, 0, 1, 2, 3, 4, 5, 0, 1, 2, 3, 4, 5, 0, 1, 2, 3, 4, 5]],
           names=[None, '日期'])